# Importazione librerie

In [2]:
import re
import spacy
from nltk.corpus import stopwords
print("Importazione librerie completata")

Importazione librerie completata


# Pipeline forum tecnico

In [3]:
class PipelineTecnica:
    def __init__(self, lingua='italian'):
        """Pipeline NLP per testi tecnici o forum"""
        self.lingua = lingua
        self.nlp = spacy.load('it_core_news_sm' if lingua == 'italian' else 'en_core_web_sm')

        # Stop words base di NLTK + rimozione di parole importanti per contesto tecnico
        stop_base = set(stopwords.words(lingua))
        # Rimuovo parole che potrebbero essere utili in contesti tecnici
        self.stop_custom = stop_base - {'errore', 'problema', 'installato', 'installare', 'modulo', 'libreria'}

    def normalizza_testo(self, testo):
        """Normalizzazione: conserva versioni, librerie, apostrofi e termini tecnici"""
        testo = testo.lower()

        # Mantieni pattern tipo 3.2.1, nomi di pacchetti, ma rimuovi punteggiatura non utile
        testo = re.sub(r'(?<!\d)[^\w\.\-\']+(?!\d)', ' ', testo)

        # Rimuovi doppi spazi
        testo = re.sub(r'\s+', ' ', testo).strip()
        return testo

    def tokenizza(self, testo):
        """Tokenizza senza lemmatizzare (preserva i termini tecnici)"""
        doc = self.nlp(testo)
        tokens = []
        for token in doc:
            if not token.is_space and not token.is_punct:
                tokens.append(token.text)
        return tokens

    def rimuovi_stop_words(self, tokens):
        """Rimuove solo stopwords non tecniche"""
        return [t for t in tokens if t not in self.stop_custom]

    def processa_testo(self, testo):
        """Pipeline completa"""
        testo_norm = self.normalizza_testo(testo)
        tokens = self.tokenizza(testo_norm)
        tokens_filtrati = self.rimuovi_stop_words(tokens)
        return tokens_filtrati


# Test

In [4]:
# Test 4 — Frammento tecnico / forum
testo4 = "Ho installato la versione 3.2.1 di TensorFlow, ma ottengo ancora l’errore ‘module not found’. Qualcuno sa come risolvere?"

pipeline_tech = PipelineTecnica()
risultato4 = pipeline_tech.processa_testo(testo4)

print("💻 Tokens finali:", risultato4)

💻 Tokens finali: ['installato', 'versione', '3.2.1', 'tensorflow', 'ottengo', 'ancora', 'errore', 'module', 'not', 'found', 'qualcuno', 'sa', 'risolvere']


# Approccio utilizzato
Nel caso di testi tecnici o frammenti da forum, l’obiettivo è preservare termini specifici e tecnicismi, evitando che la pulizia del testo elimini informazioni cruciali come nomi di librerie, versioni, messaggi di errore o comandi.

La pipeline quindi:
- mantiene numeri e versioni (es. 3.2.1),
- conserva parole con apostrofi o punti interni (come nomi di pacchetti Python),
- non esegue lemmatizzazione (per evitare distorsioni semantiche sui termini tecnici),
- usa stopwords ridotte (personalizzate per non eliminare parole utili come errore, installato, modulo).